# Exemplo copia e cola

In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
sequence_to_classify = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


{'sequence': 'Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU', 'labels': ['politics', 'economy', 'environment', 'entertainment'], 'scores': [0.9656293392181396, 0.023067045956850052, 0.00737267080694437, 0.003930921200662851]}


# Adaptando o exemplo

In [3]:
!pip install -Uq datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_

In [4]:
from datasets import load_dataset
dataset = load_dataset("celsowm/bbc_news_ptbr")

README.md:   0%|          | 0.00/638 [00:00<?, ?B/s]

(…)-00000-of-00001-97102f0adab65e78.parquet:   0%|          | 0.00/40.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8637 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['categoria', 'titulo', 'texto', 'data', 'link'],
        num_rows: 8637
    })
})

In [6]:
df = dataset['train'].to_pandas()

df

,categoria,titulo,texto,data,link
0,brasil,"Quem foi Oswaldo Aranha, o brasileiro que ajud...",“Sua atuação no cenário internacional estava d...,2023-11-04,https://www.bbc.com/portuguese/articles/c0wxwr...
1,brasil,Como PCC fez Paraguai virar um dos países com ...,"No ranking de 2023, a nação saltou para a 4ª p...",2023-11-04,https://www.bbc.com/portuguese/articles/cp646z...
2,brasil,"Vídeo, Massa ou Milei: o que Brasil tem a ganh...",O segundo turno das eleições presidenciais na ...,2023-11-02,https://www.bbc.com/portuguese/articles/ckv0gk...
3,brasil,"Dia de Finados: como celebração dos mortos, qu...","""Do Senhor é a terra e a sua plenitude, o mund...",2023-11-02,https://www.bbc.com/portuguese/articles/c1354g...
4,brasil,A estratégia do Brasil por resolução sobre Isr...,Após quatro propostas terem sido rejeitadas na...,2023-10-31,https://www.bbc.com/portuguese/articles/clw7wl...
...,...,...,...,...,...
8632,sociedade,O impacto sobre ciclistas mulheres dos assédio...,Uma ciclista que viralizou após confrontar um ...,2021-06-13,https://www.bbc.com/portuguese/internacional-5...
8633,sociedade,Britânica conta como deu soco em crocodilo par...,"Como todas as histórias de terror, a de Georgi...",2021-06-11,https://www.bbc.com/portuguese/internacional-5...
8634,sociedade,A cidade asiática obcecada com a limpeza,Sinto isso toda vez que saio do avião: o frio ...,2021-06-10,https://www.bbc.com/portuguese/vert-tra-56995398
8635,sociedade,"Vídeo, Os mistérios da vida, morte e aparência...",Eternizada no imaginário popular com a pele br...,2021-06-10,https://www.bbc.com/portuguese/geral-57427857


In [7]:
df.categoria.value_counts()

,count
categoria,
brasil,960
internacional,960
economia,960
saude,960
ciencia,960
tecnologia,960
america_latina,959
cultura,959
sociedade,959


In [11]:


categorias = df.categoria.unique()

def classifica_texto(texto,categorias=categorias):
  resultado = classifier(texto,categorias)
  resultados_ordenados = sorted(zip(resultado['labels'], resultado['scores']), key=lambda x: x[1], reverse=True)
  return resultados_ordenados[0][0]

classifica_texto("Os empregos caíram no último trimestre")

'economia'

In [17]:
from datasets import Dataset
amostra = Dataset.from_pandas(df.sample(100))

def classifica_texto(texto):
  resultado = classifier(texto['titulo'], categorias)
  resultados_ordenados = sorted(zip(resultado['labels'], resultado['scores']), key=lambda x: x[1], reverse=True)
  texto['predicted_category'] = resultados_ordenados[0][0]
  return texto

amostra = amostra.map(classifica_texto)



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
amostra.to_pandas().head()



,categoria,titulo,texto,data,link,__index_level_0__,predicted_category
0,brasil,A briga entre Pacheco e Lira que pode paralisa...,Uma disputa de poder entre o Senado Federal e ...,2023-03-24,https://www.bbc.com/portuguese/articles/c03k0l...,430,brasil
1,economia,Polícias desobstruem estradas tomadas por bols...,Polícias militares dos Estados começaram a des...,2022-11-01,https://www.bbc.com/portuguese/brasil-63466820,3325,sociedade
2,cultura,Round 6: seis coisas que a série da Netflix no...,"Round 6, um drama sul-coreano sangrento, se to...",2021-10-18,https://www.bbc.com/portuguese/geral-58952413,7282,internacional
3,saude,Por que ter alergias pode reduzir risco de con...,"No decorrer da pandemia de covid-19, descobrim...",2022-08-26,https://www.bbc.com/portuguese/geral-62672508,4631,saude
4,america_latina,"Tratados como indesejáveis pela Colômbia, 'hip...","A Ministra do Meio Ambiente, Susana Muhamad, a...",2023-11-03,https://www.bbc.com/portuguese/articles/c5173p...,962,sociedade


In [21]:
amostra

Dataset({
    features: ['categoria', 'titulo', 'texto', 'data', 'link', '__index_level_0__', 'predicted_category'],
    num_rows: 100
})

In [20]:
from sklearn.metrics import classification_report

# Get the true labels and predicted labels
y_true = amostra['categoria']
y_pred = amostra['predicted_category']

# Generate and print the classification report
report = classification_report(y_true, y_pred)
print(report)

                precision    recall  f1-score   support

america_latina       0.38      0.25      0.30        12
        brasil       0.29      0.20      0.24        10
       ciencia       0.00      0.00      0.00         7
       cultura       0.40      0.33      0.36        12
      economia       0.60      0.27      0.38        11
 internacional       0.27      0.44      0.33        16
         saude       0.44      0.50      0.47        14
     sociedade       0.06      0.10      0.07        10
    tecnologia       0.38      0.38      0.38         8

      accuracy                           0.30       100
     macro avg       0.31      0.27      0.28       100
  weighted avg       0.33      0.30      0.30       100

